### IMPORT

In [1]:
import pandas as pd
import numpy as np
import pickle
import re
import os
from os.path import join
from tqdm import tqdm
from datetime import datetime
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl import Workbook

# dtype 정의
debt_dtype = {'채무자키':str, '타채무자키':str, '담당자키':str}
account_dtype = {'채무자키':str, '계좌키':str, '타채무자키':str}
grt_dtype = {'채무자키':str, '계좌키':str, '타채무자키':str, '보증인키':str}
rehabilitation_dtype = {'채무자키':str, '계좌키':str, '분납키':str, '사건키':str}
credit_dtype = {'채무자키':str, '계좌키':str, '보증인키':str, '심의차수':str, '변제금\n수취계좌':str}
event_dtype = {'채무자키':str, '계좌키':str, '법조치키':str, '관할법원코드':str, '타채무자키':str,'타법조치키':str}

### 채권구분
- 아래에서 전체 원데이터 만들기
- 새 채권구분 만들기 : 1) 종결, 2) 조정(개회,신복이 중복되더라도 일단 문제없게), 3) 무담보 순서대로 만들기
  - (오류 체크) 개회, 신복 기준데이터 vs 차주 채무자상태 
    - (전산 미수정) 조정 vs 미반영, 면책 및 완납 vs 미반영
    - (개회신복경쟁) 개회 vs 신복 or 신복 vs 개회
    - (오등록) 확정리스트 없음 vs 개회, 신복. 다음의 경우를 포함함[폐지 및 실효 vs 미반영(여전히 개회 및 신복 *개회폐지 후 신복, 신복폐지 후 개회일 수는 있음)]

- (오류 체크) 개회, 신복 보증인상태 미반영 리스트 만들기, 중복리스트 만들키
- 개회, 신복 조정여부 칼럼 추가해서 자산정리 칼럼 만들기(계좌키에 붙이기) : 생략하지 말고 다 붙이기
- 채권구분칼럼 최초채권구분으로 변경

#### 새채무상태 : 개인, 법인, 개인회생(진행중), 개인회생(확정), 신용회복(개인), 신용회복(진행중), 신용회복(프리), 종료

In [11]:
# 개별 파일 pkl 만들기
# wd = r"D:\3.자산\전산 dataset"
# basedate = "230423"
# filename = "계좌조회새창_20230424_0918"
# x = pd.read_excel(join(wd, basedate, filename +".xlsx"), dtype={'채무자키':str, '계좌키':str, '타채무자키':str}).fillna("")
# x.to_pickle(join(wd, basedate, filename +".pkl"))

In [20]:
# 파일 읽기
# 채무자, 계좌, 보증인, 개회신복 기준데이터->차주와 보증인으로 나누 저장
wd = r"D:\3.자산\전산 dataset"
################### 예수금 지울것(채무자,계좌)
basedate = "230430"
files = ("채무자조회새창_20230502_0844", "계좌조회새창_20230502_0847", "보증인새창_20230502_0849_새보증인상태", "개인회생새창_20230502_0850_진행중_계좌키분리_기준데이터", "신용회복전체리스트_20230430_기준데이터")
# 개회의 겨우 계좌키 분리 파일로 읽을 것! 

# 파일 읽기
ext = ".pkl"  
###################

# 개회-신복 충돌 정리(기준데이터 만들 때 각 담당자가 제거하기로 함)
#### 개회제외 = {"차주" : ("20462830"), "보증인" : () } # 20462830 중복입금건. 신복이 기간이 더 남아서 신복으로 처리하기로 함
#### 신복제외 = {"차주" : ("20451534", "20525880", "20525532"), "보증인" : ()} # "20451534", "20525880", "20525532" 신복 실효 확정 전에 개회 신청한 건
if ext == ".pkl" : # fillna()는 엑셀읽을 때 미리 해둬야함, 조정, 개회 진행중 시트만 읽어서 저장한 경우
    debt = pd.read_pickle(join(wd, basedate, files[0]+ext))
    account = pd.read_pickle(join(wd, basedate, files[1]+ext))
    grt = pd.read_pickle(join(wd, basedate, files[2]+ext))
    rehabilitation = pd.read_pickle(join(wd, basedate, files[3]+ext))
    credit = pd.read_pickle(join(wd, basedate, files[4]+ext))
else : 
    debt = pd.read_excel(join(wd, basedate, files[0]+ext), dtype=debt_dtype).fillna("")
    account = pd.read_excel(join(wd, basedate, files[1]+ext), dtype=account_dtype).fillna("")
    grt = pd.read_excel(join(wd, basedate, files[2]+ext), dtype=grt_dtype).fillna("")
    rehabilitation = pd.read_excel(join(wd, basedate, files[3]+ext), sheet_name="개인회생(진행)", dtype=rehabilitation_dtype).fillna("")
    credit = pd.read_excel(join(wd, basedate, files[4]+ext), sheet_name="확정,미확정", dtype=credit_dtype).fillna("")

# 엑셀저장시 NaT 를 ""로
credit.replace({pd.NaT: ""}, inplace=True)
# 조정채권 차주, 보증인 나누기
rehabilitation_d = rehabilitation[rehabilitation["분납자관계"]=="채무자"]
rehabilitation_grt = rehabilitation[rehabilitation["분납자관계"]=="보증인"]
credit_d = credit[credit["채무구분"]=="주채무"]
credit_grt = credit[credit["채무구분"]=="보증채무"]

# 예수금 제외
debt.drop(debt[debt.성명=="예수금"].index, inplace=True)
debt.reset_index(drop=True, inplace=True)
account.drop(account[account.채무자명=="예수금"].index, inplace=True)
account.reset_index(drop=True, inplace=True)

In [102]:
# 모든 날짜 포맷 컬럼 str 변경
# date_cols = credit_grt.select_dtypes(include='datetime').columns
# credit_grt[date_cols] = credit_grt[date_cols].astype(str)
# credit_grt.to_excel(join(wd, basedate, "xxyyx.xlsx"))

C:\Users\SL\AppData\Local\Temp\ipykernel_39560\2739606122.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  credit_grt[date_cols] = credit_grt[date_cols].astype(str)


In [23]:
# # pkl 저장
# debt.to_pickle(join(wd, basedate, files[0]+".pkl"))
# account.to_pickle(join(wd, basedate, files[1]+".pkl"))
# grt.to_pickle(join(wd, basedate, files[2]+".pkl"))
# rehabilitation.to_pickle(join(wd, basedate, files[3]+".pkl"))
# credit.to_pickle(join(wd, basedate, files[4]+".pkl"))

In [21]:
# 수정목록
# 개회-신복 채무자키 중복 : 차주, 보증인별

# 차주
조정중복_차주 = pd.merge(rehabilitation_d[["채무자키","계좌키","인가/미인가","반영OPB", "접수일"]], credit_d[["채무자키","계좌키","진행구분","현재원금잔액","접수일자"]], on=["채무자키","계좌키"], how="inner")
display("차주", 조정중복_차주)

# 보증인
조정중복_보증인 = pd.merge(rehabilitation_grt[["채무자키","계좌키","인가/미인가","반영OPB", "접수일"]], credit_grt[["채무자키","계좌키","진행구분","현재원금잔액", "접수일자"]], on=["채무자키","계좌키"], how="inner")
display("보증인", 조정중복_보증인)

'차주'

,채무자키,계좌키,인가/미인가,반영OPB,접수일,진행구분,현재원금잔액,접수일자


'보증인'

,채무자키,계좌키,인가/미인가,반영OPB,접수일,진행구분,현재원금잔액,접수일자


In [22]:
# [check] 계좌종결건 조정채권 포함여부
종결된계좌키1 = account[account.종결일 != ""].계좌키
종결된계좌키2 = account[account.메모 != ""].계좌키
종결된계좌키 = pd.concat([종결된계좌키1, 종결된계좌키2])
check개회 = 종결된계좌키[종결된계좌키.isin(rehabilitation["계좌키"])]
check신복 = 종결된계좌키[종결된계좌키.isin(credit["계좌키"])]
check = pd.concat([check개회, check신복])
if len(check) > 0 :
    print(check)
    # 출력
    # check.to_excel(join(wd, basedate, "9.종결vs조정.xlsx"), index=False)
    # 종결건 신복과 개회에서 제외하기
    rehabilitation = rehabilitation[~rehabilitation.계좌키.isin(check)]
    credit = credit[~credit.계좌키.isin(check)]
else : print("이상무")

10896    200930335
21725    200941164
38437    200957876
5165     200924604
5166     200924605
12374    200931813
13395    200932834
17944    200937383
19071    200938510
19494    200938933
20220    200939659
27450    200946889
28296    200947735
Name: 계좌키, dtype: object


In [23]:
# 확정된 조정채권 저장하기
rehabilitation_d = rehabilitation[rehabilitation["분납자관계"]=="채무자"]
rehabilitation_grt = rehabilitation[rehabilitation["분납자관계"]=="보증인"]
credit_d = credit[credit["채무구분"]=="주채무"]
credit_grt = credit[credit["채무구분"]=="보증채무"]

In [24]:
# 개회 >> 차주 : 계좌키, 보증인 : [계좌키 + 보증인이름]이 pk. 
# 신복 >> 보증인키 있으므로 이걸로. 누락계좌는 정상으로 나옴
# 개회 누락채권(일부계좌누락, 일부 채무자키(이런 경우가 있을까?)) 문제는 어떻게? : 일단 

# 보증인 상태변경
if "새보증인상태" in grt.columns : 
    grt["새보증인상태"] = ""
else : grt.insert(8,"새보증인상태","")

# '채무자키'열과 '보증인성명인'열을 기준으로 비교하여 값 변경
for i in tqdm(range(len(grt)), total=len(grt)):  
    # 종결건 보증인상태 유지
    
    # 개인회생
    temp = rehabilitation_grt[(rehabilitation_grt['채무자키'] == grt.at[i, '채무자키']) & (rehabilitation_grt['분납자성명인'] == grt.at[i, '보증인성명인'])]
    if len(temp) > 0:
        grt.at[i, '새보증인상태'] = "개인회생(확정)" if temp['인가/미인가'].values[0] == "인가" else "개인회생(진행중)"
    else : 
        # 신용회복
        temp = credit_grt[(credit_grt['보증인키'] == grt.at[i, '보증인키'])]
        if len(temp) > 0:
            grt.at[i, '새보증인상태'] = "신용회복("+ temp['상환방식'].values[0]+")" if temp['진행구분'].values[0] == "확정" else "신용회복(진행중)"
        else :
            # 무담보
            grt.at[i, '새보증인상태'] = grt.at[i, '보증인상태']

# move = grt.pop('새보증인상태')
# grt.insert(8,'새보증인상태', move)

100%|██████████| 10312/10312 [00:04<00:00, 2249.44it/s]


In [7]:
# 저장 : 보증인 수정할 것 엑셀 저장
grt[grt.새보증인상태 != grt.보증인상태].loc[:,["채무자키", "계좌키","보증인키","매각사구분","보증인성명인","주민번호인","계좌번호","보증구분","새보증인상태","보증인상태","보증금액","보증잔액","보증한도액"]].to_excel(join(wd, basedate, "9.보증인상태변경할것.xlsx"), index=False)
# # 새보증인상태 추가한 파일 저장
# grt.to_pickle(join(wd, basedate, files[2]+"_새보증인상태.pkl"))
grt.to_excel(join(wd, basedate, files[2]+"_새보증인상태.xlsx"), index=False)

In [25]:
# 계좌에 새채무상태 만들기

if "새채무상태" not in account.columns : account.insert(8,"새채무상태", "")
종결처리안된종료건 = []

for i in tqdm(range(len(account)), total = len(account)):
    account_key = account.at[i, '계좌키']
    # 종결처리 안 된 계좌(종결일 없는 것)
    if account.at[i, '종결일'] == "" : # 종결일 있는경우 메모는 자유롭게 적어도 되겠다. but 종결일 없는 경우는 종료 새채무상태의 분류에 따라
        # 종결일 없는 것 중 메모에 새채무상태 적어 놓은 것 그대로 옮기기 
        if account.at[i, "메모"] != "" :
            account.at[i, '새채무상태'] = account.at[i, "메모"]
            # checkList에 추가
            종결처리안된종료건.append(account.at[i, '계좌키'])
        # 종결일도 없고, 메모도 빈칸(미종결) : 개회 > 신복 > 나머지는 나중에 채움
        else : 
            # 개인회생
            temp = rehabilitation_d[(rehabilitation_d['계좌키'] == account_key)]
            if  len(temp) > 0:
                account.at[i, '새채무상태'] = "개인회생(확정)" if temp['인가/미인가'].values[0] == "인가" else "개인회생(진행중)"
            else : 
                # 신용회복
                temp = credit_d[(credit_d['계좌키'] == account.at[i, '계좌키'])]
                if len(temp) > 0:
                    account.at[i, '새채무상태'] = "신용회복("+ temp['상환방식'].values[0]+")" if temp['진행구분'].values[0] == "확정" else "신용회복(진행중)"
                else :
                    # 일단 빈칸으로 두고 뒤에서 수정 # 무담보 또는 종료(면책)
                    account.at[i, '새채무상태'] = ""
    # 종결처리 된 계좌
    else :
        account.at[i, '새채무상태'] = "종료"

# move = account.pop('새보증인상태')
# account.insert(8,'새보증인상태', move)

100%|██████████| 78157/78157 [01:13<00:00, 1058.39it/s]


In [26]:
# [ckeck]종결처리해야하는 계좌 출력
if len(종결처리안된종료건)>0 :
    # print(종결처리안된종료건)
    pd.DataFrame(종결처리안된종료건).to_excel(join(wd, basedate, "9.종결처리안된종료건.xlsx"), index=False)
else : print("이상무")

이상무


### 전체 원데이터 만들기(종합)

In [27]:
raw_data = pd.DataFrame(None)
raw_data["채무자키"] = account.채무자키
raw_data["계좌키"] = account.계좌키
raw_data["타채무자키"] = account.타채무자키

raw_data["채권구분"] = account.채권구분
raw_data["새채무상태"] = account.새채무상태
raw_data["채무상태"] = account.채무상태
raw_data["담당자"] = account.담당자
raw_data["성명"] = account.채무자명

raw_data["주민등록번호"] = account.채무자주민번호 #.apply(lambda x : x[:8])
raw_data["계좌번호"] = account.계좌번호
raw_data["원채권사"] = account.원채권사
raw_data["매각사구분"] = account.매각사구분
raw_data["계정과목"] = account.계정과목
raw_data["최초원금"] = account.최초원금
raw_data["최초법비용"] = account.최초법비용
raw_data["최초미수이자"] = account.최초미수이자
raw_data["최초연체이자율"] = account.최초연체이자율
raw_data["현재원금"] = account.현재원금
raw_data["현재법비용"] = account.현재법비용
raw_data["OPB"] = account[["현재원금","현재법비용"]].sum(axis=1)
raw_data["현재미수이자"] = account.현재미수이자
raw_data["현재연체이자"] = account.현재연체이자
raw_data["현재합계"] = account.현재합계
raw_data["회수합계"] = account.회수합계
raw_data["최초대출일"] = account.최초대출일
raw_data["대출만기일"] = account.대출만기일
raw_data["최초연체일"] = account.최초연체일 # 없는 경우도 있음 없으면 대출만기일로 대체해야.
raw_data["매입일"] = account.매입일
# 시효관련 추가
raw_data['시효완성월'] = np.where((account['시효중단여부'] == 'Y'), '중단', account['시효완성월'])
raw_data["시효사유"] = account.시효연장사유 # 시효연장사유 빈칸인 경우, 최초완성일메모(거의), 최초시효완성일, 시효중단여부 모두 빈칸임
raw_data["시효완성일"] = account.시효완성일

# 채무자조회에서 불러올 항목
debt_temp = debt[["채무자키", "개인법인", "연령", "자택우편번호", "자택주소"]]
raw_data = pd.merge(raw_data, debt_temp, on="채무자키", how="left")

# grt에서 각 계좌별 보증인상태 불러오기
names = grt.groupby('계좌키')['보증인성명인'].apply(lambda x: ','.join(x)) # 고유값만 남기려면 join의 매개변수로 x.unique()를 사용
status = grt.groupby('계좌키')['보증인상태'].apply(lambda x: ','.join(x))

raw_data = raw_data.merge(names, how='left', on='계좌키')
raw_data = raw_data.merge(status, how='left', on='계좌키')

raw_data["보증인성명인"] = raw_data["보증인성명인"].fillna('')
raw_data["보증인상태"] = raw_data["보증인상태"].fillna('')

In [28]:
# 새채무상태 마무리
print("전",raw_data.새채무상태.value_counts())
채무상태종료 = re.compile("종결|환매|매각|추심제외")
담당자종료 = re.compile("종결|매각|매각예정|회계파산|DS|환매|포기채권")
채무상태면책 = re.compile("개인회생\(면책\)|파산\(면책\)")
보증인종료 = re.compile("파산\(면책\)|사망|종결|보증면탈|개인회생\(면책\)|사망\(상속포기\)|시효완성|시효|추심제외|폐업|매각|환매")

for i in tqdm(range(len(raw_data)), total = len(raw_data)):
    if raw_data.at[i, "새채무상태"] == "" :
        # 종료
        if 채무상태종료.search(raw_data.at[i, "채무상태"]) or 담당자종료.search(raw_data.at[i, "담당자"]) :
            raw_data.at[i, "새채무상태"] = "종료"
        # 차주면책
        elif 채무상태면책.search(raw_data.at[i, "채무상태"]) :
            # 보증인 없으면
            if raw_data.at[i, "보증인상태"] == "" :
                raw_data.at[i, "새채무상태"] = "종료"
            else : # 보증인종료 이외의 건이 있는지
                temp = 보증인종료.sub("",raw_data.at[i, "보증인상태"])
                if len(re.sub(",", "", temp)) == 0 : # 보증인도 모두 종료/면책
                    raw_data.at[i, "새채무상태"] = "종료"
                else :
                    raw_data.at[i, "새채무상태"] = re.sub("사업자", "", raw_data.at[i, "개인법인"])
        else :
            raw_data.at[i, "새채무상태"] = re.sub("사업자", "", raw_data.at[i, "개인법인"])
print("후",raw_data.새채무상태.value_counts())

전              43926
종료           11939
신용회복(개인)      9189
개인회생(확정)      6048
신용회복(프리)      5942
신용회복(진행중)      658
개인회생(진행중)      455
Name: 새채무상태, dtype: int64


100%|██████████| 78157/78157 [00:03<00:00, 24997.91it/s]

후 개인           32717
종료           19284
신용회복(개인)      9189
개인회생(확정)      6048
신용회복(프리)      5942
법인            3864
신용회복(진행중)      658
개인회생(진행중)      455
Name: 새채무상태, dtype: int64


In [65]:
# [ckeck]계좌별 채무상태 수정할 것 엑셀 저장
# 범주가 다르게 적용되어서 현재는 의미가 없다. 보완이 필요
# raw_data[raw_data.새채무상태 != raw_data.채무상태].loc[:,["채무자키", "계좌키","매각사구분","새채무상태","채무상태"]].to_excel(join(wd, basedate, "9.계좌별채무상태달라진것.xlsx"), index=False)

In [29]:
# 최종OPB 작성하기
개회OPB = rehabilitation_d.loc[:, ["계좌키", "반영OPB"]].rename(columns={"반영OPB" : "최종OPB"})
신복OPB = credit_d.loc[:, ["계좌키", "현재원금잔액"]].rename(columns={"현재원금잔액" : "최종OPB"})
신복OPB["최종OPB"] = 신복OPB["최종OPB"].astype(int)
# 개회OPB와 신복OPB 병합
merged_opb = 개회OPB.merge(신복OPB, on=['계좌키','최종OPB'], how='outer')
# raw_data와 병합
raw_data = raw_data.merge(merged_opb, on='계좌키', how='outer')
# 조정이 아닌 경우, 종료이면 0, 아니면 OPB
new_col = np.where(raw_data['새채무상태'] == '종료', 0, raw_data['OPB'])
raw_data.loc[raw_data['최종OPB'].isnull(), '최종OPB'] = new_col[raw_data['최종OPB'].isnull()]

In [30]:
# 전체 저장하기
raw_data.to_excel(join(wd, basedate, "2.솔림헬프_전체_원데이터_"+basedate+".xlsx"), index=False)
# 특정 매각사만 저장하기
# cond = "스마트저축-01" ###################################
# raw_data[raw_data.매각사구분==cond].to_excel(join(wd, cond+"_원데이터_"+basedate+".xlsx"), index=False)

In [96]:
# [check] 더블 채무상태
채무자별채무상태 = raw_data.groupby('주민등록번호')['새채무상태'].apply(lambda x:",".join(x.unique()))
check_채무상태 = 채무자별채무상태[채무자별채무상태.str.contains(',')]
# 출력
if len(check_채무상태) > 0 : check_채무상태.to_excel(join(wd, basedate, "9.더블채무상태.xlsx"))

In [98]:
# 제출용 시트 작성하기-개인회생 
re_d = rehabilitation_d.loc[:,["채무자키","계좌키","분납키","분납자성명인","분납자관계","매각사구분", "인가/미인가","현재결과", "반영OPB", "관할법원", "사건번호", "접수일", "개시결정일","변제계획인가일","총분납회차","현재회차","납입회차","잔여회차","미납회차","총분납금","총분납입금","분납미납금","분납잔금","총분납잔금","현재원금","현재법비용","비고"]]
re_grt = rehabilitation_grt.loc[:,["채무자키","계좌키","분납키","분납자성명인","분납자관계","매각사구분", "인가/미인가","현재결과", "반영OPB", "관할법원", "사건번호", "접수일", "개시결정일","변제계획인가일","총분납회차","현재회차","납입회차","잔여회차","미납회차","총분납금","총분납입금","분납미납금","분납잔금","총분납잔금","현재원금","현재법비용","비고"]]
# re_d.insert(0, "순번", range(1,len(re_d)+1))
# re_grt.insert(0, "순번", range(1,len(re_grt)+1))
# 저장
with pd.ExcelWriter(join(wd, basedate, "2.솔림헬프_개인회생_원데이터_"+basedate+".xlsx"), engine='openpyxl', mode = 'w') as writer :
    re_d.to_excel(writer, sheet_name="차주", index=False)
    re_grt.to_excel(writer, sheet_name="보증인", index=False)

In [104]:
# 제출용 시트 작성하기-신용회복
# 기준데이터에서 삭제한 것이 있을 때 유의미 & 차주 보증인 구분

# 날짜열 문자열로
date_cols = credit_d.select_dtypes(include='datetime').columns
credit_d.loc[:,date_cols] = credit_d[date_cols].astype(str)
date_cols = credit_grt.select_dtypes(include='datetime').columns
credit_grt.loc[:,date_cols] = credit_grt[date_cols].astype(str)

# 저장
with pd.ExcelWriter(join(wd, basedate, "2.솔림헬프_신용회복_원데이터_"+basedate+".xlsx"), engine='openpyxl', mode = 'w') as writer :
    credit_d.to_excel(writer, sheet_name="차주", index=False)
    credit_grt.to_excel(writer, sheet_name="보증인", index=False)

In [100]:
# 요약 
pd.options.display.float_format = '{:.0f}'.format
# 특정 POOL
# pool계좌키 = pd.read_excel("c://users/sl/Desktop/계좌키.xlsx", dtype={"계좌키":str})
# pool = pd.merge(raw_data,pool계좌키, on='계좌키', how='inner')
# result = pool.groupby('새채무상태').agg({'계좌키':'count', '최종OPB':'sum'}).reindex(index=['개인','법인','개인회생(확정)','개인회생(진행중)','신용회복(개인)','신용회복(프리)','신용회복(진행중)','종료'])
# result
# 전체
raw_data.groupby('새채무상태').agg({'계좌키':'count', '최종OPB':'sum'}).reindex(index=['개인','법인','개인회생(확정)','개인회생(진행중)','신용회복(개인)','신용회복(프리)','신용회복(진행중)','종료'])

,계좌키,최종OPB
새채무상태,,
개인,32717,270489324790
법인,3864,293279746341
개인회생(확정),6048,14011977004
개인회생(진행중),455,3015099407
신용회복(개인),9189,40882709363
신용회복(프리),5942,35244968063
신용회복(진행중),658,4261072432
종료,19284,0


### POOL별 엑셀 파일 만들기

In [2]:
# 원데이터 읽기

# # 바로 작업
# 무담보 = raw_data[raw_data["새채무상태"].isin(["개인","법인"])].iloc[:,:-1]

# 새로 읽기
wd = r"D:\3.자산\전산 dataset\230430\매각환매 채무상태 수정후"
basedate = "230430"
files = ["2.솔림헬프_개인회생_원데이터_230430", "2.솔림헬프_신용회복_원데이터_230430", "2.솔림헬프_전체_원데이터_230430"]

# 1.개인회생
re_d = pd.read_excel(join(wd, files[0]+".xlsx"), dtype=rehabilitation_dtype, sheet_name="차주").fillna("")
re_grt = pd.read_excel(join(wd, files[0]+".xlsx"), dtype=rehabilitation_dtype, sheet_name="보증인").fillna("")

# 2.신용회복
credit_d = pd.read_excel(join(wd, files[1]+".xlsx"), dtype=credit_dtype, sheet_name="차주").fillna("")
credit_grt = pd.read_excel(join(wd, files[1]+".xlsx"), dtype=credit_dtype, sheet_name="보증인").fillna("")

# NaT값을 ""
credit_d.replace({pd.NaT: ""}, inplace=True)
credit_grt.replace({pd.NaT: ""}, inplace=True)

# 날짜타입 칼럼
# date_cols = credit_d.select_dtypes(include='datetime').columns
# credit_d.loc[:,date_cols] = credit_d[date_cols].astype(str)
# date_cols = credit_grt.select_dtypes(include='datetime').columns
# credit_grt.loc[:,date_cols] = credit_grt[date_cols].astype(str)

# 3.무담보
raw_data = pd.read_excel(join(wd, files[2]+".xlsx"), dtype=account_dtype, sheet_name="리스트").fillna("")

In [3]:
###################################################
# 무담보 채무상태 추가작업
raw_data.loc[raw_data.채무상태.str.contains("파산|사망"),"새채무상태"] = "종료"
###################################################

In [4]:
# 풀데이터 읽기
pooldata = pd.read_excel(r"D:\3.자산\POOL별 관리자산\전체\0.전체정리_230331_채권구분&POOL표시.xlsx", sheet_name="POOL정리", dtype={"채무자키":str})

In [5]:
# POOL별 채무자키
pool_dict = {
            # "POOL1" : pooldata.query('P1==True')["채무자키"],
            # "POOL2" : pooldata.query('P2==True')["채무자키"],
            # "POOL3" : pooldata.query('P3==True')["채무자키"],
            # "POOL4" : pooldata.query('P4==True')["채무자키"],
            # "POOL5" : pooldata.query('P5==True')["채무자키"],
            # "POOL6" : pooldata.query('P6==True')["채무자키"],
            # "POOL7" : pooldata.query('P7==True')["채무자키"],
            # "POOL8" : pooldata.query('P8==True')["채무자키"],
            # "POOL9" : pooldata.query('P9==True')["채무자키"],
            # "POOL10" : pooldata.query('P10==True')["채무자키"],
            # "POOL11" : pooldata.query('P11==True')["채무자키"],
            # "POOL12" : pooldata.query('P12==True')["채무자키"],
            # "POOL13" : pooldata.query('P13==True')["채무자키"],
            # "POOL14" : pooldata.query('P14==True')["채무자키"],
            # "POOL15" : pooldata.query('P15==True')["채무자키"],
            "A1" : pooldata.query('A1==True')["채무자키"]
            }

##### 풀별 내부/발송 파일 만들기
- 일부 풀만 작업할 경우 위 pool_dict 주석처리를 통해

In [6]:
pd.set_option('mode.chained_assignment', None)

for pool_kind, pool_keys in tqdm(pool_dict.items(), total=len(pool_dict))  : 
    
    ###### 내부용 만들기############################################################
    file_kind = "내부"
    # 풀 원데이터
    pool_raw = raw_data[raw_data.채무자키.isin(pool_keys)]
    # 풀 무담보, 종료건
    pool_무담보 = pool_raw[pool_raw["새채무상태"].isin(["개인","법인"])].iloc[:,:-1] # 무담보는 opb = 최종opb니까
    pool_종료 = pool_raw[pool_raw["새채무상태"]=="종료"].iloc[:,:-1]
    # 풀 조정건
    pool_re_d = re_d[re_d.채무자키.isin(pool_keys)]
    pool_re_grt = re_grt[re_grt.채무자키.isin(pool_keys)]
    pool_credit_d = credit_d[credit_d.채무자키.isin(pool_keys)]
    pool_credit_grt = credit_grt[credit_grt.채무자키.isin(pool_keys)]

    #순번 넣기
    pool_종료.insert(0, "순번", range(1,len(pool_종료)+1))
    pool_무담보.insert(0, "순번", range(1,len(pool_무담보)+1))
    pool_re_d.insert(0, "순번", range(1,len(pool_re_d)+1))
    pool_re_grt.insert(0, "순번", range(1,len(pool_re_grt)+1))
    pool_credit_d["No."] = range(1,len(pool_credit_d)+1)
    pool_credit_grt["No."] = range(1,len(pool_credit_grt)+1)

    ####### 요약시트[시작]
    summary = pool_raw.groupby('새채무상태').agg({'계좌키':'count', '최종OPB':'sum'}).reindex(index=['개인','법인','개인회생(확정)','개인회생(진행중)','신용회복(개인)','신용회복(프리)','신용회복(진행중)','종료'])
    summary.rename(columns={'계좌키':'건수', '최종OPB':"OPB"}, inplace=True)

    # 보증인 합산
    summary["건수(보증인)"] = [
        0,0, # 무담보는 그대로
        pool_re_grt[pool_re_grt['인가/미인가']=="인가"].계좌키.count(), # 보증인은 없는 경우 있어서 그룹화하면 index에러날 수 있음
        pool_re_grt[pool_re_grt['인가/미인가']=="미인가"].계좌키.count(),
        pool_credit_grt[(pool_credit_grt.진행구분=="확정") & (pool_credit_grt.상환방식=="개인")].계좌키.count(),
        pool_credit_grt[(pool_credit_grt.진행구분=="확정") & (pool_credit_grt.상환방식=="프리")].계좌키.count(),
        pool_credit_grt[pool_credit_grt.진행구분=="미확정"].계좌키.count(), 0
    ]
    summary["OPB(보증인)"] = [
        0,0, # 무담보는 그대로
        pool_re_grt[pool_re_grt['인가/미인가']=="인가"].반영OPB.sum(),
        pool_re_grt[pool_re_grt['인가/미인가']=="미인가"].반영OPB.sum(),
        pool_credit_grt[(pool_credit_grt.진행구분=="확정") & (pool_credit_grt.상환방식=="개인")].OPB.sum(),
        pool_credit_grt[(pool_credit_grt.진행구분=="확정") & (pool_credit_grt.상환방식=="프리")].OPB.sum(),
        pool_credit_grt[pool_credit_grt.진행구분=="미확정"].OPB.sum(), 0
    ]

    # "종결" 행 제외한 열의 합계 계산
    row_sum = summary[summary.index != "종료"].sum()
    summary.loc["계(종료제외)"] = row_sum
    summary = summary.reindex(index=['개인','법인','개인회생(확정)','개인회생(진행중)','신용회복(개인)','신용회복(프리)','신용회복(진행중)',"계(종료제외)",'종료'])

    summary = summary.fillna(0).astype(dtype='int64')
    summary.rename_axis("구분", inplace=True)
    

    ######### 엑셀 쓰기
    wb = Workbook()
    ws1 = wb.active
    ws1.title = "요약"
    ws2 = wb.create_sheet("무담보")
    ws3 = wb.create_sheet("개인회생_차주")
    ws4 = wb.create_sheet("개인회생_보증인")
    ws5 = wb.create_sheet("신용회복_차주")
    ws6 = wb.create_sheet("신용회복_보증인")
    ws7 = wb.create_sheet("종결")

    # 시트에 데이터 쓰기
    for r in dataframe_to_rows(summary, index=True, header=True): ws1.append(r)
    # a1에 구분 넣고 요약셀 2행 삭제
    ws1['a1'] = "구분"
    ws1.delete_rows(2)
    for r in dataframe_to_rows(pool_무담보, index=False, header=True): ws2.append(r)
    for r in dataframe_to_rows(pool_re_d, index=False, header=True): ws3.append(r)
    for r in dataframe_to_rows(pool_re_grt, index=False, header=True): ws4.append(r)
    for r in dataframe_to_rows(pool_credit_d, index=False, header=True): ws5.append(r)
    for r in dataframe_to_rows(pool_credit_grt, index=False, header=True): ws6.append(r)
    for r in dataframe_to_rows(pool_종료, index=False, header=True): ws7.append(r)

    # 스타일 지정
    # 폰트
    #font = Font(name='NN30', size=10, color='FF000000')
    # 폰트 (데이터 부분)
    font_data = Font(name='NN30', size=10)
    # 배경색
    fill_col = PatternFill(fill_type='solid', start_color='FF0072C6', end_color='FF0072C6')
    fill_new = PatternFill(fill_type='solid', start_color='FF49C620', end_color='FF49C620')
    # 테두리
    border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
    # 정렬
    alignment = Alignment(horizontal='center', vertical='center')

    for ws in [ws1,ws2,ws3,ws4,ws5,ws6,ws7] :

        # 컬럼명 스타일 적용
        for cell in ws[1]:
            cell.font = Font(name='NN30', size=10, color='FFFFFF', bold=True)
            if cell.value is not None and cell.value.startswith('새') : cell.fill = fill_new
            else : cell.fill = fill_col
            cell.border = border
            cell.alignment = alignment

        # 데이터 부분 스타일 적용
        for row in ws.iter_rows(min_row=2):
            for cell in row:
                # 숫자 데이터인 경우
                if isinstance(cell.value, (int, float)):
                    cell.font = font_data
                    cell.number_format = '#,##0' if isinstance(cell.value, int) else '#,##0.00'
                    cell.alignment = Alignment(horizontal='right', vertical='center')
                # 날짜 데이터인 경우
                elif isinstance(cell.value, pd.Timestamp):
                    cell.font = font_data
                    cell.number_format = 'yyyy-mm-dd'
                    cell.alignment = alignment
                # 나머지 문자열 데이터인 경우
                else:
                    cell.font = font_data
                    cell.alignment = alignment
                cell.border = border

    # 요약시트 "계(종료제외)" 배경색
    for cell in ws1[9]:
        cell.fill = PatternFill(fill_type='solid', start_color='B8CCE4', end_color='B8CCE4')

    # 파일 저장
    wb.save(join(wd, pool_kind+"_데이터_"+basedate+"_"+file_kind+".xlsx"))


    ###### 발송용 만들기############################################################
    file_kind = "발송" ####

    pool_무담보.drop(['새채무상태','채무상태','담당자'], axis=1, inplace=True)
    pool_무담보["성명"] = pool_무담보["성명"].apply(lambda x:x[:1]+"\u25CB"+x[2:])
    pool_무담보["주민등록번호"] = pool_무담보["주민등록번호"].apply(lambda x : x[:8]+"*"*len(x[8:]))
    pool_re_d["분납자성명인"] = pool_re_d["분납자성명인"].apply(lambda x:x[:1]+"\u25CB"+x[2:])
    pool_re_grt["분납자성명인"] = pool_re_grt["분납자성명인"].apply(lambda x:x[:1]+"\u25CB"+x[2:])
    pool_credit_d["고객명"] = pool_credit_d["고객명"].apply(lambda x:x[:1]+"\u25CB"+x[2:])
    pool_credit_d["고객식별번호"] = pool_credit_d["고객식별번호"].apply(lambda x : x[:8]+"*"*len(x[8:]))
    pool_credit_grt["고객명"] = pool_credit_grt["고객명"].apply(lambda x:x[:1]+"\u25CB"+x[2:])
    pool_credit_grt["고객식별번호"] = pool_credit_grt["고객식별번호"].apply(lambda x : x[:8]+"*"*len(x[8:]))
    pool_종료 = pool_종료[["순번","채무자키","계좌키","채무상태","성명","주민등록번호"]]
    pool_종료["성명"] = pool_종료["성명"].apply(lambda x:x[:1]+"\u25CB"+x[2:])
    pool_종료["주민등록번호"] = pool_종료["주민등록번호"].apply(lambda x : x[:8]+"*"*len(x[8:]))

    wb = Workbook()
    ws1 = wb.active
    ws1.title = "요약"
    ws2 = wb.create_sheet("무담보")
    ws3 = wb.create_sheet("개인회생_차주")
    ws4 = wb.create_sheet("개인회생_보증인")
    ws5 = wb.create_sheet("신용회복_차주")
    ws6 = wb.create_sheet("신용회복_보증인")
    ws7 = wb.create_sheet("종결")

    # 시트에 데이터 쓰기
    for r in dataframe_to_rows(summary, index=True, header=True): ws1.append(r)
    for r in dataframe_to_rows(pool_무담보, index=False, header=True): ws2.append(r)
    for r in dataframe_to_rows(pool_re_d, index=False, header=True): ws3.append(r)
    for r in dataframe_to_rows(pool_re_grt, index=False, header=True): ws4.append(r)
    for r in dataframe_to_rows(pool_credit_d, index=False, header=True): ws5.append(r)
    for r in dataframe_to_rows(pool_credit_grt, index=False, header=True): ws6.append(r)
    for r in dataframe_to_rows(pool_종료, index=False, header=True): ws7.append(r)

    # 스타일 지정
    # 폰트
    #font = Font(name='NN30', size=10, color='FF000000')
    # 폰트 (데이터 부분)
    font_data = Font(name='NN30', size=10)
    # 배경색
    fill = PatternFill(fill_type='solid', start_color='FF0072C6', end_color='FF0072C6')
    # 테두리
    border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
    # 정렬
    alignment = Alignment(horizontal='center', vertical='center')

    for ws in [ws1,ws2,ws3,ws4,ws5,ws6,ws7] :

        # 컬럼명 스타일 적용
        for cell in ws[1]: # 시트의 1행의 셀 반복
            cell.font = Font(name='NN30', size=10, color='FFFFFF', bold=True)
            cell.fill = fill
            cell.border = border
            cell.alignment = alignment

        # 데이터 부분 스타일 적용
        for row in ws.iter_rows(min_row=2):
            for cell in row:
                # 숫자 데이터인 경우
                if isinstance(cell.value, (int, float)):
                    cell.font = font_data
                    cell.number_format = '#,##0' if isinstance(cell.value, int) else '#,##0.00'
                    cell.alignment = Alignment(horizontal='right', vertical='center')
                # 날짜 데이터인 경우
                elif isinstance(cell.value, pd.Timestamp):
                    cell.font = font_data
                    cell.number_format = 'yyyy-mm-dd'
                    cell.alignment = alignment
                # 나머지 문자열 데이터인 경우
                else:
                    cell.font = font_data
                    cell.alignment = alignment
                cell.border = border
        
    # 요약시트 "계(종료제외)" 배경색
    for cell in ws1[10]:
        cell.fill = PatternFill(fill_type='solid', start_color='B8CCE4', end_color='B8CCE4')

    # 파일 저장
    wb.save(join(wd, pool_kind+"_데이터_"+basedate+"_"+file_kind+".xlsx"))


100%|██████████| 1/1 [00:01<00:00,  1.57s/it]


In [ ]:
############## 끝 ###################

### 인적속성 - 다중채무, 다중담당 체크하기

In [301]:
####전산_check_전체계좌.ipynb 에도 있음
account = pd.read_pickle(r"D:\3.자산\전산 dataset\230430\계좌조회새창_20230502_0847.pkl")
new_debtKey = pd.read_pickle(r"C:\Users\SL\Desktop\workspace\python\fileNaming\파일\새채무자키.pkl")[["새채무자키","채무자키","계좌키"]]
# 새채무자키 병합하기
account.drop(account[account.채무자명=="예수금"].index, inplace=True)
account = pd.merge(account, new_debtKey[["계좌키","새채무자키",]], on='계좌키', how='left')

# 필요열만 새로운 df로 & 정렬
new_debtKey = account[["새채무자키", '채무자키','계좌키','채무상태','담당자']].copy()
new_debtKey.sort_values(['새채무자키', '채무자키', '계좌키'], inplace=True)

###### multi_index : 조작하기는 어렵고, 엑셀로 보낼때 자동으로 병합이 되어서 보기는 좋음
x = account.set_index(['새채무자키','채무자키','계좌키'],drop=True)["채무상태"]
# 다중채무, 다중담당 T/F값 입력
x = new_debtKey.groupby('새채무자키')
new_debtKey['다중채무상태여부'] = x['채무상태'].transform(lambda s: np.any(s != s.iloc[0]))
new_debtKey['다중담당여부'] = x['담당자'].transform(lambda s: np.any(s != s.iloc[0]))
# 다중인 것들만 
check = new_debtKey[new_debtKey["다중채무상태여부"] | new_debtKey['다중담당여부']].copy()
# index설정
gc = check.set_index(['새채무자키', '채무자키'], drop=True)
#저장
gc.to_excel(r"c://Users/SL/Desktop/check_채무상태(병합).xlsx")

### 시효(계좌)

In [14]:
wd = r"C:\Users\SL\Desktop\자산정리test파일"
filename_account = "계좌조회새창_20230502_0847.pkl"
PATH_NEW_DEBTKEY = r"C:\Users\SL\Desktop\자산정리test파일\새채무자키.pkl"

account = pd.read_pickle(join(wd,filename_account))
new_debtKey = pd.read_pickle(PATH_NEW_DEBTKEY)
today = datetime.today().strftime("%y%m%d")

In [15]:
# 새채무자키 병합하기
account.drop(account[account.채무자명=="예수금"].index, inplace=True)
account = pd.merge(account, new_debtKey[["계좌키","새채무자키",]], on='계좌키', how='left')

In [ ]:
# 필요열만 새로운 df로 & 정렬
account_limit_info = account[["새채무자키", '채무자키','계좌키','채무상태','담당자', "시효완성일", "시효완성월", "최초연체일", "최초시효완성일메모", "시효연장사유", "시효중단여부"]].copy()
account_limit_info.sort_values(['새채무자키', '채무자키', '계좌키'], inplace=True)

In [ ]:
# 동일한 새채무자키 시효중단여부 : 시효가 ##########################################작성중 중단
def set_extension_info(group):
    if 'Y' in group['시효중단여부'].values:
        group.loc[group['시효중단여부'] == '', '시효중단여부'] = 'Y'
        extension_reason = group.loc[group['시효중단여부'] == 'Y', '시효연장사유'].values[0]
        group['시효연장사유'] = extension_reason
    return group

account_limit_info = account_limit_info.groupby('새채무자키').apply(set_extension_info)


### 시효(법조치)

In [8]:
wd = r"C:\Users\SL\Desktop\자산정리test파일"
filename_event = "법조치조회새창_20230522_2054.xlsx"
dtype_event = {"채무자키":str, "계좌키":str, "법조치키":str}
today = datetime.today().strftime("%y%m%d")

In [9]:
# 파일 읽기/필요칼럼추리기/원본파일pkl과 필요컬럼xlsx 파일 저장하기
df_event_ori = pd.read_excel(join(wd, filename_event), dtype=dtype_event)

c:\Users\SL\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
df_event = pd.DataFrame(None, columns=["채무자키", "계좌키", "법조치키", "법조치구분", "법조치세부", "관할법원", "사건번호", "사건구분", "접수일", "확정일", "종료일",
    "법조치상태", "종국결과", "청구금액법원", "청구금액", "관련법조치관할법원", "관련법조치사건번호"])
    ###################################################### 여기까지 하고 계좌시트 작업으로 넘어감
df_event.채무자키 = df_event_ori.채무자키
df_event.계좌키 = df_event_ori.계좌키
df_event.신청일자 = df_event_ori.접수일
df_event.법무구분 = df_event_ori.법조치구분
df_event.관할법원 = df_event_ori.관할법원
df_event.사건번호 = df_event_ori.사건번호.str.replace(" ", "")
df_event.사건구분 = df_event_ori.사건구분
df_event.진행상태 = df_event_ori.종국결과
df_event.청구금액법원 = df_event_ori.청구금액법원
df_event.청구금액 = df_event_ori.청구금액
print(len(df_event.index))

####################
# 중복제거전 저장
df_event.to_pickle(join(wd, "전처리완료", "전처리_법조치_중복미제거.pkl"))
#df_event.to_excel(join(wd, "전처리완료", "전처리_법조치_중복미제거.xlsx"), index=False)
####################

# 채무자키로 전 계좌 불러오기
work_all_account = all_df[all_df.채무자키.isin(work_df.채무자키)].copy() # 객체 새로 만들지 않으면 계속 copywarning 나옴
# 전 계좌 중 작업 계좌에 있거나, 종결된 계좌면 Y, 아니면 N(솔림 잔존계좌)
cond = (work_all_account.계좌키.isin(work_df.계좌키)) | (work_all_account.종결일 != "")
work_all_account["계좌작업대상여부"] = np.where(cond, "Y", "N")

# 채무자키별로 전계좌 여부 확인 후 파일 저장하기
result = pd.DataFrame(None, columns=["채무자키", "전계좌여부", "채무상태", "담당자", "채무자명"])
temp = work_all_account.drop_duplicates('채무자키').copy()
result["채무자키"] = temp.채무자키.values
result["채무상태"] = temp.채무상태.values # 순서 조작만 하지 않으면 굳이 일치조건 하지 않아도 됨.
result["담당자"] = temp.담당자.values
result["채무자명"] = temp.채무자명.values


# 작업대상여부의 유니크값이 1이면 모두 Y (모두 N인 경우는 논리적으로 불가능), 2면 작업계좌와 잔존계좌가 혼재한 것이므로 일부계좌
g = work_all_account.groupby('채무자키') # 그룹화는 원본 df의 순서와 다를 수 있음!!!!!!!!
for i, v in result.iterrows() : 
     if g.get_group(v.채무자키).계좌작업대상여부.nunique() == 1 :
          result.loc[i, "전계좌여부"]="전계좌"
     else : result.loc[i, "전계좌여부"]="일부계좌"

# 저장하기
result.to_excel(path_save, index=False)